In [1]:
using ITensors
ITensors.compile
using Plots 
using GR

In [2]:
# we define global lattice data structure such that MPOs share indicies
N=4    

# assign N ℂ^2 vector spaces 
lat = siteinds("S=1/2", N);

In [3]:
function Πuu(i::Int64)#, N::Int64) 
    # s=1; ms=1 quantum numbers
    # states span (i, i+1) sites 
    opsum = OpSum()
    add!(opsum,"ProjUp",i,"ProjUp",mod(i,N)+1)
    return MPO(opsum,lat)
end

Πuu (generic function with 1 method)

In [4]:
function Π10(i::Int64)#, N::Int64) 
    # s=1; ms=0 quantum numbers
    # states span (i, i+1) sites 
    opsum = OpSum()
    add!(opsum,"ProjUp",i,"ProjDn",mod(i,N)+1)
    opsum += ("ProjDn",i,"ProjUp",mod(i,N)+1)
    opsum += ("S+",i,"S-",mod(i,N)+1) # cross terms
    opsum += ("S-",i,"S+",mod(i,N)+1) # cross terms
    return MPO(opsum,lat)
end

Π10 (generic function with 1 method)

In [5]:
function Πdd(i::Int64)#, N::Int64) 
    # s=1; ms=-1 quantum numbers
    # states span (i, i+1) sites 
    opsum = OpSum()
    add!(opsum,"ProjDn",i,"ProjDn",mod(i,N)+1)
    return MPO(opsum,lat)
end

Πdd (generic function with 1 method)

In [6]:
function Π00(i::Int64)# , N::Int64) 
    # s=0; ms=0 quantum numbers: singlet
    # states span (i, i+1) sites 
    opsum = OpSum()
    add!(opsum,"ProjUp",i,"ProjDn",mod(i,N)+1)
    opsum += ("ProjDn",i,"ProjUp",mod(i,N)+1)
    opsum -= ("S+",i,"S-",mod(i,N)+1) # cross terms
    opsum -= ("S-",i,"S+",mod(i,N)+1) # cross terms
    return MPO(opsum,lat) 
end

Π00 (generic function with 1 method)

In [7]:
function Π(i,s,m)
    if s==1 && m==1
        return Πuu(i)
    elseif s==1 && m==0
        return Π10(i)
    elseif s==1 && m==-1
        return Πdd(i)
    elseif s==0 && m==0
        return Π00(i)
    end
end

Π (generic function with 1 method)

In [8]:
function Π(i,s,m)
    if s==1 && m==1
        return Πuu(i)
    elseif s==1 && m==0
        return Π10(i)
    elseif s==1 && m==-1
        return Πdd(i)
    elseif s==0 && m==0
        return Π00(i)
    end
end

Π (generic function with 1 method)

In [9]:
operators = Array{Any}(nothing, 2^N)
sm = [(0,0),(1,0),(1,1),(1,-1)]
for n = 1:2^N
    i = div(n-1,N) + 1
    r = rem(n-1,N) + 1
    operators[n] = Π(i,sm[r][1],sm[r][2])
    print(i,'\t',r,'\t',sm[r],'\n')
end

1	1	(0, 0)
1	2	(1, 0)
1	3	(1, 1)
1	4	(1, -1)
2	1	(0, 0)
2	2	(1, 0)
2	3	(1, 1)
2	4	(1, -1)
3	1	(0, 0)
3	2	(1, 0)
3	3	(1, 1)
3	4	(1, -1)
4	1	(0, 0)
4	2	(1, 0)
4	3	(1, 1)
4	4	(1, -1)


In [11]:
println(tr(apply(operators[3], operators[8];)))
for i=1:4
    for j=i:4
        println(tr(apply(operators[i], operators[j];)))
    end
end

2.195711598545798e-16
16.0
4.7874356540981414e-15
3.546046371670331e-16
1.3078184159238113e-15
16.0
3.546046371670331e-16
1.3078184159238113e-15
4.0000000000000036
0.0
4.000000000000002
